In [28]:
#Modelo 1 
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# 1. Descargar datos históricos del activo
activo = "AAPL"  # Ticker del activo
periodo = "1y"
datos = yf.Ticker(activo).history(period=periodo)

# 2. Calcular los retornos promedio (Average) y la desviación estándar (std)
datos['Average'] = datos[['High', 'Low']].mean(axis=1).pct_change()
datos['std'] = datos['Average'].rolling(window=5).std()

# Eliminar valores NaN para los ajustes
datos.dropna(inplace=True)

# --- MODELO POLINOMIAL ---
# Ajustar un modelo polinomial de grado 2
X_poly = np.column_stack((datos['Average'], datos['Average']**2))  # [x, x^2]
X_poly = sm.add_constant(X_poly)  # Añadir intercepto
modelo_poly = sm.OLS(datos['std'], X_poly).fit()

# Coeficientes del modelo polinomial
alpha_poly = modelo_poly.params[0]
beta_1_poly = modelo_poly.params[1]
beta_2_poly = modelo_poly.params[2]

# Predicciones del modelo polinomial
datos['std_pred_poly'] = alpha_poly + beta_1_poly * datos['Average'] + beta_2_poly * datos['Average']**2

# --- MODELO EXPONENCIAL ---
# Definir la función exponencial
def modelo_exponencial(x, alpha, beta):
    return alpha * np.exp(beta * x)

# Ajustar el modelo exponencial
popt, _ = curve_fit(modelo_exponencial, datos['Average'], datos['std'], maxfev=10000)
alpha_exp, beta_exp = popt

# Predicciones del modelo exponencial
datos['std_pred_exp'] = modelo_exponencial(datos['Average'], alpha_exp, beta_exp)

# --- GRAFICAR LOS RESULTADOS ---
fig = go.Figure()

# Datos observados
fig.add_trace(go.Scatter(x=datos['Average'], y=datos['std'],
                         mode='markers', name='Datos Observados',
                         marker=dict(size=8, color='blue', opacity=0.7)))

# Línea del modelo polinomial
x_vals = np.linspace(datos['Average'].min(), datos['Average'].max(), 100)
y_vals_poly = alpha_poly + beta_1_poly * x_vals + beta_2_poly * x_vals**2
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_poly, mode='lines', name='Modelo Polinomial (Grado 2)',
                         line=dict(color='red')))

# Línea del modelo exponencial
y_vals_exp = modelo_exponencial(x_vals, alpha_exp, beta_exp)
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_exp, mode='lines', name='Modelo Exponencial',
                         line=dict(color='green')))

# Configuración del gráfico
fig.update_layout(title="Modelos Polinomial y Exponencial | Desv. Estándar vs Retornos Promedio",
                  xaxis_title="Retornos Promedio (Average)",
                  yaxis_title="Desviación Estándar (std)",
                  template="plotly_white")

fig.show()

# --- RESULTADOS ---
print("Resultados del Modelo Polinomial:")
print(f"Intercepto (α): {alpha_poly:.4f}")
print(f"Coeficiente Lineal (β1): {beta_1_poly:.4f}")
print(f"Coeficiente Cuadrático (β2): {beta_2_poly:.4f}")
print(f"Resumen del Modelo Polinomial:\n{modelo_poly.summary()}")

print("\nResultados del Modelo Exponencial:")
print(f"Intercepto (α): {alpha_exp:.4f}")
print(f"Coeficiente Exponencial (β): {beta_exp:.4f}")


C:\Users\retir\AppData\Local\Temp\ipykernel_15772\2765941249.py:28: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_15772\2765941249.py:29: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\retir\AppData\Local\Temp\ipykernel_15772\2765941249.py:30: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Resultados del Modelo Polinomial:
Intercepto (α): 0.0102
Coeficiente Lineal (β1): -0.0115
Coeficiente Cuadrático (β2): 5.0564
Resumen del Modelo Polinomial:
                            OLS Regression Results                            
Dep. Variable:                    std   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     24.27
Date:                Sun, 26 Jan 2025   Prob (F-statistic):           2.45e-10
Time:                        22:43:25   Log-Likelihood:                 883.92
No. Observations:                 246   AIC:                            -1762.
Df Residuals:                     243   BIC:                            -1751.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t